# Tesla specific time series processing
The goal of this notebook is to demonstrate the implementation of time series processing steps that are specific to Tesla.

## Setup

### Imports

In [ ]:
import plotly.express as px

from core.pandas_utils import *
from transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries
from transform.raw_results.tesla_results import get_results

### Data extraction

In [ ]:
! mkdir -p data_cache

In [ ]:
tss = TeslaProcessedTimeSeries()

In [ ]:
tss.to_parquet("data_cache/tesla_tss.parquet")

In [ ]:
TARGET_VIN = "LRW3E7FA4MC314534"

## Charging masksing and indexing

In [ ]:
def compute_in_charge_idx(tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=False)
    shifted_vars = tss_grp[["in_charge", "charge_energy_added"]].shift(fill_value=False)
    tss["new_charge_period_mask"] = shifted_vars["in_charge"].ne(tss["in_charge"]) 
    tss["new_charge_period_mask"] |= (
        shifted_vars["charge_energy_added"].gt(tss["charge_energy_added"])
        & shifted_vars["in_charge"]
        & tss["in_charge"]
    )
    tss["in_charge_idx"] = tss_grp["new_charge_period_mask"].cumsum().astype("uint16")
    tss = tss.drop(columns=["new_charge_period_mask"])
    return tss

def trim_leading_n_trailing_soc_off_masks(tss:DF, masks:list[str]) -> DF:
    for mask in masks:
        mask_idx = mask + "_idx"
        tss_grp = tss.groupby(["vin", mask_idx], observed=False)
        trailing_soc = tss_grp["soc"].transform("last")
        leading_soc = tss_grp["soc"].transform("first")
        tss["trailing_soc"] = trailing_soc
        tss["leading_soc"] = leading_soc
        tss[f"trimmed_{mask}"] = tss[mask] & (tss["soc"] != trailing_soc) & (tss["soc"] != leading_soc)
    return tss

def compute_idx_from_masks(tss: DF, masks:list[str], max_td:pd.Timedelta) -> DF:
    for mask in masks:
        idx_col_name = f"{mask}_idx"
        shifted_mask = tss.groupby("vin", observed=False)[mask].shift(fill_value=False)
        tss["new_period_start_mask"] = shifted_mask.ne(tss[mask]) 
        if max_td is not None:
            tss["new_period_start_mask"] |= tss["time_diff"].gt(max_td) & tss[mask]
        tss[idx_col_name] = tss.groupby("vin", observed=False)["new_period_start_mask"].cumsum().astype("uint16")
        tss.drop(columns=["new_period_start_mask"], inplace=True)
    return tss

def fill_uncertain_charge_periods(tss:DF, in_charge_mask:str="in_charge", in_discharge_mask:str="in_discharge") -> DF:
    tss_grp = tss.groupby(["vin"], observed=False)
    tss["nan_charge"] = tss[in_charge_mask].mask(~tss[in_charge_mask] & ~tss[in_discharge_mask], pd.NA).astype("boolean")
    ffill_charge = tss_grp["nan_charge"].ffill()
    bfill_charge = tss_grp["nan_charge"].bfill()
    tss["nan_charge"] = ffill_charge.where(ffill_charge == bfill_charge, pd.NA).astype("boolean")
    tss["nan_charge"] = tss["nan_charge"].mask(tss["soc"] >= 98, pd.NA)

    tss["new_nan_charge_period"] = tss_grp["nan_charge"].shift().ne(tss["nan_charge"]) 
    tss["nan_charge_idx"] = tss_grp["new_nan_charge_period"].cumsum().fillna(0)
    tss = tss.drop(columns=["new_nan_charge_period"])
    return tss


In [ ]:
ts = tss.query("vin == @TARGET_VIN").copy()

In [ ]:
ts:DF = (
    ts
    .eval("in_charge_below_98 = in_charge & soc < 98")
    .pipe(compute_in_charge_idx)
    .pipe(trim_leading_n_trailing_soc_off_masks, ["in_charge"])
    .pipe(fill_uncertain_charge_periods, "in_charge_below_98")
    .pipe(compute_idx_from_masks, ["trimmed_in_charge"], TD(hours=1, minutes=30))
)

In [ ]:
px.scatter(
    ts.eval("in_charge_str = nan_charge.astype('string').fillna('unknown')"),
    x="date",
    y="soc",
    color="in_charge_str",
    symbol="nan_charge_idx",
    hover_data=[
        "in_charge_str",
        "trimmed_in_charge",
        "sec_time_diff",
        "charge_energy_added",
        "charging_status",
        "leading_soc",
        "trailing_soc",
    ],
).update_layout(showlegend=False)

In [ ]:
ts.query("nan_charge == True").groupby("nan_charge_idx").ngroups

In [ ]:
results = get_results()

In [ ]:
results.groupby("vin")["soh"].count().sort_values()

In [ ]:
MANY_CHARGES_VIN = "LRW3E7FA4MC314534"
many_charges_ts = tss.query("vin == @MANY_CHARGES_VIN")

In [ ]:
px.scatter(
    many_charges_ts,
    x="date",
    y='soc',
    color="trimmed_in_charge",
    symbol="trimmed_in_charge_idx",
)

In [ ]:
THIBAULT_VIN = "5YJ3E7EB7KF474436"

In [ ]:
thibault_ts = tss.query("vin == @THIBAULT_VIN")
thibault_ts["charge_limit_soc"].value_counts(dropna=False)

In [ ]:
thibault_ts:DF = (
    thibault_ts
    .eval("in_charge_below_98 = in_charge & soc < 98")
    .pipe(compute_in_charge_idx)
    .pipe(trim_leading_n_trailing_soc_off_masks, ["in_charge"])
    .pipe(fill_uncertain_charge_periods, "in_charge_below_98")
    .pipe(compute_idx_from_masks, ["trimmed_in_charge"], TD(hours=1, minutes=30))
)

In [ ]:
display(thibault_ts.query("nan_charge == True")["nan_charge_idx"].nunique())
display(thibault_ts.query("trimmed_in_charge")["trimmed_in_charge_idx"].nunique())

In [ ]:
px.scatter(
    thibault_ts.eval("in_charge_str = nan_charge.astype('string').fillna('unknown')"),
    x="date",
    y="soc",
    color="in_charge_str",
    symbol="nan_charge_idx",
    hover_data=[
        "in_charge_str",
        "trimmed_in_charge",
        "sec_time_diff",
        "charge_energy_added",
        "charging_status",
        "leading_soc",
        "trailing_soc",
    ],
).update_layout(showlegend=True)

In [ ]:
px.scatter(
    thibault_ts.eval("in_charge_str = nan_charge.astype('string').fillna('unknown')"),
    x="date",
    y="soc",
    color="trimmed_in_charge",
    symbol="trimmed_in_charge_idx",
    hover_data=[
        "in_charge_str",
        "trimmed_in_charge",
        "sec_time_diff",
        "charge_energy_added",
        "charging_status",
        "leading_soc",
        "trailing_soc",
    ],
).update_layout(showlegend=True)

In [ ]:
thibault_ts.melt(id_vars=["vin", "date"], value_vars=["soc", "charge_limit_soc"])

In [ ]:
thibault_ts["charge_limit_soc_std"].value_counts()

In [ ]:
thibault_ts.dtypes

In [ ]:
px.scatter(
    (
        thibault_ts
        .eval(" = odometer")
        .eval("test = charge_limit_soc + charge_limit_soc_std / 10")
        .melt(id_vars=["vin", "date"], value_vars=["soc", "test", "odometer"])
    ),
    x="date",
    y="value",
    color="variable"
)

In [ ]:
tss.dtypes